In [2]:
import MeCab as mc
import re
import pandas as pd
import numpy as np
import urllib.request
import unicodedata
from sklearn.model_selection import train_test_split

In [3]:
#tweet dataの読み込み
tweet_input = pd.read_csv(filepath_or_buffer="Datas/all_data/tweet_datas.csv", encoding="utf_8", sep=",")
print(len(tweet_input))

58481


In [4]:
tweet_input.info

<bound method DataFrame.info of                 screen_name     user_id      tweet_id  \
0             岩手県宮古市公式アカウント  2459550828  6.714788e+17   
1                       花巻市  2843129221  6.714900e+17   
2                       花巻市  2843129221  6.714906e+17   
3                       花巻市  2843129221  6.714914e+17   
4                       花巻市  2843129221  6.714924e+17   
5                  イーハトーブログ   196899097  6.714973e+17   
6                奥州市公式アカウント   264482835  6.715166e+17   
7             岩手県宮古市公式アカウント  2459550828  6.715203e+17   
8      遠野市観光協会/Tono Tourist  2324801065  6.715214e+17   
9                       花巻市  2843129221  6.715248e+17   
10     遠野市観光協会/Tono Tourist  2324801065  6.715386e+17   
11                 イーハトーブログ   196899097  6.715485e+17   
12       さんてつくん @ 三陸鉄道【鉄ログ】   133582577  6.715527e+17   
13                      花巻市  2843129221  6.715580e+17   
14                    遠野テレビ    85263819  6.715611e+17   
15                    遠野テレビ    85263819  6.715614e+17   

In [5]:
#NaNデータのカウント
print(tweet_input.isnull().sum())
#NaNデータの中身を確認
#tweet_input[tweet_input['tweet'].isnull()]
#NaNのデータを削除
tweet_data = tweet_input.dropna(how='any')
#NaNデータのカウント
print(tweet_data.isnull().sum())
#掲載したツイート数のカウント
published_post = tweet_data['retweet'] == 1
published_post.sum()

screen_name      1
user_id          1
tweet_id         1
tweet          549
postdate         7
retweet          8
dtype: int64
screen_name    0
user_id        0
tweet_id       0
tweet          0
postdate       0
retweet        0
dtype: int64


10079

In [6]:
tweet_data.head()

,screen_name,user_id,tweet_id,tweet,postdate,retweet
0,岩手県宮古市公式アカウント,2459550828,6.714788e+17,広報12月1号を市ホームページにアップしました（）。本号では「宮古港開港400周年」の特集記...,2015-12-01 00:00:00,0.0
1,花巻市,2843129221,6.714900e+17,自動証明写真機設置を希望する事業者を募集します。\n≪申込受付期間≫\n平成27年12月2日...,2015-12-01 00:00:00,0.0
2,花巻市,2843129221,6.714906e+17,【固定資産税に関する各種申告等について】\n固定資産税は、毎年1月1日現在の資産の所有状況に...,2015-12-01 00:00:00,0.0
3,花巻市,2843129221,6.714914e+17,【広報はなまき 平成27年12月1日号】\n 広報はなまき12月1日号を発行しました。\n...,2015-12-01 00:00:00,0.0
4,花巻市,2843129221,6.714924e+17,【こどものつどい「親子で揚げよう！成島和紙で凧づくり体験」参加者募集】\n郷土に伝わる成島和...,2015-12-01 00:00:00,0.0


In [7]:
#不要な文字の削除
tweet_data["tweet"] = tweet_data["tweet"].str.replace('\n', ' ').str.replace('_nan', ' ').str.replace(r'[0-9]', ' ').str.replace(r'([^\s\w])+', ' ')

/Users/swallowtail/.local/share/virtualenvs/classification_tweet-zASJ7jKU/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [8]:
tweet_data.head()

,screen_name,user_id,tweet_id,tweet,postdate,retweet
0,岩手県宮古市公式アカウント,2459550828,6.714788e+17,広報 月 号を市ホームページにアップしました 本号では 宮古港開港 周年 の特集記事を...,2015-12-01 00:00:00,0.0
1,花巻市,2843129221,6.714900e+17,自動証明写真機設置を希望する事業者を募集します 申込受付期間 平成 年 月 日 水...,2015-12-01 00:00:00,0.0
2,花巻市,2843129221,6.714906e+17,固定資産税に関する各種申告等について 固定資産税は 毎年 月 日現在の資産の所有状況によ...,2015-12-01 00:00:00,0.0
3,花巻市,2843129221,6.714914e+17,広報はなまき 平成 年 月 日号 広報はなまき 月 日号を発行しました ...,2015-12-01 00:00:00,0.0
4,花巻市,2843129221,6.714924e+17,こどものつどい 親子で揚げよう 成島和紙で凧づくり体験 参加者募集 郷土に伝わる成島和紙...,2015-12-01 00:00:00,0.0


In [99]:
#特定のデータ数のカウント
test = ((tweet_data['screen_name'] == '川源ぶどう園(花巻市)') & (tweet_data['retweet'] == 1.0)) 
print(test.sum())

4


In [10]:
tweet_data.to_csv("Datas/all_data/all_tweet.csv",index=False, sep=",")

In [11]:
#postdateをdatetime形式に変換
print(pd.to_datetime(tweet_data['postdate'], format = '%Y-%m-%d'))

0       2015-12-01
1       2015-12-01
2       2015-12-01
3       2015-12-01
4       2015-12-01
5       2015-12-01
6       2015-12-01
7       2015-12-01
8       2015-12-01
9       2015-12-01
10      2015-12-01
11      2015-12-01
12      2015-12-01
13      2015-12-01
14      2015-12-01
15      2015-12-01
16      2015-12-01
17      2015-12-01
18      2015-12-01
19      2015-12-01
20      2015-12-01
21      2015-12-01
22      2015-12-01
23      2015-12-01
24      2015-12-01
25      2015-12-02
26      2015-12-02
27      2015-12-02
28      2015-12-02
29      2015-12-02
           ...    
58451   2019-11-28
58452   2019-11-28
58453   2019-11-28
58454   2019-11-28
58455   2019-11-28
58456   2019-11-28
58457   2019-11-28
58458   2019-11-28
58459   2019-11-28
58460   2019-11-29
58461   2019-11-29
58462   2019-11-29
58463   2019-11-29
58464   2019-11-29
58465   2019-11-29
58466   2019-11-29
58467   2019-11-29
58468   2019-11-29
58469   2019-11-29
58470   2019-11-29
58471   2019-11-29
58472   2019

In [85]:
#テキストデータのみ取り出し、ファイルに書き出す
text = data_s.drop(['tweetID', 'retweet', 'date_x'], axis=1)
text.to_csv("tweet.txt",header=False, index=False, sep=",")
#retweetのラベルのみ取り出す
tweet_labe = data_s.drop(['tweetID', 'date_x', 'tweet2'], axis=1)
tweet_labe.to_csv("label.txt", header=Falsem index=False, sep=",")

In [51]:
#ツイートのテキスト読み込み
test = open("tweet.txt", "r")
lines = test.readlines()
test.close()
print(len(lines))

10000


In [4]:
#テキストのクリーニング
#text_data_test  = lines[9924:9929]
#print(text_data_test)
def clean_text(text_string):
    text_string = " ".join(text_string.split())
    text_string = text_string.lower()
    return(text_string)

#text_data_train = [clean_text(x) for x in text_data_test]
#print(text_data_train)

In [5]:
#slothlibのストップワードの取得
def get_stopword():
    slothlib_path = 'http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt'
    slothlib_file = urllib.request.urlopen(slothlib_path)
    slothlib_stopwords = [line.decode("utf-8").strip() for line in slothlib_file]
    slothlib_stopwords = [ss for ss in slothlib_stopwords if not ss==u'']
    return slothlib_stopwords

In [6]:
stopwords = get_stopword()
print(stopwords)

['あそこ', 'あたり', 'あちら', 'あっち', 'あと', 'あな', 'あなた', 'あれ', 'いくつ', 'いつ', 'いま', 'いや', 'いろいろ', 'うち', 'おおまか', 'おまえ', 'おれ', 'がい', 'かく', 'かたち', 'かやの', 'から', 'がら', 'きた', 'くせ', 'ここ', 'こっち', 'こと', 'ごと', 'こちら', 'ごっちゃ', 'これ', 'これら', 'ごろ', 'さまざま', 'さらい', 'さん', 'しかた', 'しよう', 'すか', 'ずつ', 'すね', 'すべて', 'ぜんぶ', 'そう', 'そこ', 'そちら', 'そっち', 'そで', 'それ', 'それぞれ', 'それなり', 'たくさん', 'たち', 'たび', 'ため', 'だめ', 'ちゃ', 'ちゃん', 'てん', 'とおり', 'とき', 'どこ', 'どこか', 'ところ', 'どちら', 'どっか', 'どっち', 'どれ', 'なか', 'なかば', 'なに', 'など', 'なん', 'はじめ', 'はず', 'はるか', 'ひと', 'ひとつ', 'ふく', 'ぶり', 'べつ', 'へん', 'ぺん', 'ほう', 'ほか', 'まさ', 'まし', 'まとも', 'まま', 'みたい', 'みつ', 'みなさん', 'みんな', 'もと', 'もの', 'もん', 'やつ', 'よう', 'よそ', 'わけ', 'わたし', 'ハイ', '上', '中', '下', '字', '年', '月', '日', '時', '分', '秒', '週', '火', '水', '木', '金', '土', '国', '都', '道', '府', '県', '市', '区', '町', '村', '各', '第', '方', '何', '的', '度', '文', '者', '性', '体', '人', '他', '今', '部', '課', '係', '外', '類', '達', '気', '室', '口', '誰', '用', '界', '会', '首', '男', '女', '別', '話', '私', '屋', '店', '家', '場', '等', '見', '際', '観', '段', '

In [7]:
#MeCabによる形態素解析
def extractter(text):
    text = unicodedata.normalize("NFKC", text)
    tagger = mc.Tagger('-Ochasen')
    tagger.parse('')
    node = tagger.parseToNode(text)
    key= []
    word = []
    keyword =[]
    while node:
        nes = node.feature.split(",")
        wor = node.surface
        if wor != "":
            nes = node.feature.split(",")
            if nes[0] == u"名詞":
                if nes[6] == u"*":
                     keyword.append(wor)
                else:
                    keyword.append(nes[6])
        node = node.next
        if node is None:
            break
    return keyword

In [8]:
text = '【祝】北海道新幹線が開業 | 2016年3月26日(土) -    北海道の皆さん、全国の皆さん、新幹線で岩手におでんせ～♪あっ、飛行機もアルヨ‼'
text = clean_text(text)
result = extractter(text)
print(result)

['祝', '北海道', '新幹線', '開業', '年月', '日土', '北海道', '皆さん', '全国', '皆さん', '新幹線', '岩手', 'おでん', '飛行機', 'アルヨ']


In [141]:
#ストップワードの除去
def except_stopwords(text, stopwords):
    for i in text:
        for j in stopwords:
            if i == j:
                text.remove(j)
    return text

In [142]:
s_result = except_stopwords(result, stopwords)
print(s_result)

['祝', '北海道新幹線', '開業', '年月', '日土', '北海道', '皆さん', '全国', '皆さん', '新幹線', '岩手', 'おでん', 'する', '飛行機', 'アル']


In [ ]:
#データセットの分割
x_train, x_test, y_train, y_test = train_test_split(namelist2_x, namelist2_y, test_size=0.3)